### Домашнее задание
1. скачать набор данных маркетинговых кампаний отсюда https://www.kaggle.com/davinwijaya/customer-retention
2. там поле conversion - это целевая переменная, а offer - коммуникация. Переименовать поля (conversion -> target, offer -> treatment) и привести поле treatment к бинарному виду (1 или 0, т.е было какое-то предложение или нет) - значение No Offer означает отсутствие коммуникации, а все остальные - наличие.
3. сделать разбиение набора данных не тренировочную и тестовую выборки
4. сделать feature engineering на ваше усмотрение (допускается свобода выбора методов)
5. провести uplift-моделирование 3 способами: одна модель с признаком коммуникации (S learner), модель с трансформацией таргета (трансформация классов п. 2. 1) и вариант с двумя независимыми моделями
6. в конце вывести единую таблицу сравнения метрик uplift@10%, uplift@20% этих 3 моделей
7. построить модель UpliftTreeClassifier и попытаться описать словами полученное дерево
8. (опционально) для модели S learner (модель с дополнительным признаком коммуникации) построить зависимость таргета (конверсии - поле conversion) от значения uplift: 1) сделать прогноз и получить uplift для тестовой выборки 2) отсортировать тестовую выборку по uplift по убыванию 3) разбить на децили (pandas qcut вам в помощь) 4) для каждого дециля посчитать среднюю conversion
9. (опционально) построить модель UpliftRandomForestClassifier и попытаться описать словами полученное дерево
Ссылки
https://towardsdatascience.com/a-quick-uplift-modeling-introduction-6e14de32bfe0
https://habr.com/ru/company/ru_mts/blog/485980/#reference1
https://en.wikipedia.org/wiki/Uplift_modelling
https://www.youtube.com/watch?v=yFQAIJBYXI0
https://www.youtube.com/watch?v=jCUcYiBK03I
https://www.uplift-modeling.com/en/latest/
https://arxiv.org/pdf/1809.04559.pdf
https://catboost.ai/docs/concepts/about.html
Библиотеки и пакеты
causalml
sklift
catboost

In [41]:
%matplotlib inline
import pandas as pd
import numpy as np


df = pd.read_csv("data.csv")
df.head(5)

,recency,history,used_discount,used_bogo,zip_code,is_referral,channel,offer,conversion
0,10,142.44,1,0,Surburban,0,Phone,Buy One Get One,0
1,6,329.08,1,1,Rural,1,Web,No Offer,0
2,7,180.65,0,1,Surburban,1,Web,Buy One Get One,0
3,9,675.83,1,0,Rural,1,Web,Discount,0
4,2,45.34,1,0,Urban,0,Web,Buy One Get One,0


In [2]:
df.rename(columns={'conversion': 'target', 'offer': 'treatment'}, inplace=True)

       recency  history  used_discount  used_bogo   zip_code  is_referral  \
0           10   142.44              1          0  Surburban            0   
1            6   329.08              1          1      Rural            1   
2            7   180.65              0          1  Surburban            1   
3            9   675.83              1          0      Rural            1   
4            2    45.34              1          0      Urban            0   
...        ...      ...            ...        ...        ...          ...   
63995       10   105.54              1          0      Urban            0   
63996        5    38.91              0          1      Urban            1   
63997        6    29.99              1          0      Urban            1   
63998        1   552.94              1          0  Surburban            1   
63999        1   472.82              0          1  Surburban            0   

            channel        treatment  target  
0             Phone  Buy One

In [5]:
df.head(20)

,recency,history,used_discount,used_bogo,zip_code,is_referral,channel,treatment,target
0,10,142.44,1,0,Surburban,0,Phone,Buy One Get One,0
1,6,329.08,1,1,Rural,1,Web,No Offer,0
2,7,180.65,0,1,Surburban,1,Web,Buy One Get One,0
3,9,675.83,1,0,Rural,1,Web,Discount,0
4,2,45.34,1,0,Urban,0,Web,Buy One Get One,0
5,6,134.83,0,1,Surburban,0,Phone,Buy One Get One,1
6,9,280.20,1,0,Surburban,1,Phone,Buy One Get One,0
7,9,46.42,0,1,Urban,0,Phone,Buy One Get One,0
8,9,675.07,1,1,Rural,1,Phone,Discount,0
9,10,32.84,0,1,Urban,1,Web,Buy One Get One,0


In [4]:
res = pd.get_dummies(treatment)

NameError: name 'treatment' is not defined

In [6]:
df['treatment']= df['treatment'].replace(['Buy One Get One'], 1)
df['treatment']= df['treatment'].replace(['Discount'], 1)
df['treatment'] =df['treatment'].replace(['No Offer'], 0)

In [7]:
df.head(6)

,recency,history,used_discount,used_bogo,zip_code,is_referral,channel,treatment,target
0,10,142.44,1,0,Surburban,0,Phone,1,0
1,6,329.08,1,1,Rural,1,Web,0,0
2,7,180.65,0,1,Surburban,1,Web,1,0
3,9,675.83,1,0,Rural,1,Web,1,0
4,2,45.34,1,0,Urban,0,Web,1,0
5,6,134.83,0,1,Surburban,0,Phone,1,1


In [13]:
probs = np.random.rand(len(df))
training_mask = probs < 0.7
test_mask = (probs>=0.7)

In [14]:
df_train = df[training_mask]
df_test = df[test_mask]

In [15]:
df_train.groupby("history")['target'].describe()

,count,mean,std,min,25%,50%,75%,max
history,,,,,,,,
29.99,5532.0,0.118764,0.323539,0.0,0.0,0.0,0.0,1.0
30.00,2.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
30.01,3.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
30.02,1.0,0.000000,NaN,0.0,0.0,0.0,0.0,0.0
30.03,1.0,0.000000,NaN,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
2816.01,1.0,0.000000,NaN,0.0,0.0,0.0,0.0,0.0
2859.63,1.0,0.000000,NaN,0.0,0.0,0.0,0.0,0.0
3040.20,1.0,0.000000,NaN,0.0,0.0,0.0,0.0,0.0


In [19]:
!pip install scikit-uplift==0.2.0

In [22]:
!pip install catboost

  Created wheel for retrying: filename=retrying-1.3.3-py3-none-any.whl size=11435 sha256=c11799ffcfe856fb62afbc45196ea8d9bcdf8960e8e478fbfc3303b18ef42c53
  Stored in directory: c:\users\матвей\appdata\local\pip\cache\wheels\c4\a7\48\0a434133f6d56e878ca511c0e6c38326907c0792f67b476e56
Successfully built retrying


In [27]:
from sklearn.model_selection import train_test_split

In [47]:
X_train, X_test, y_train, y_test = train_test_split(df, df['history'], random_state=0)

In [52]:
from sklift.metrics import uplift_at_k
from sklift.viz import plot_uplift_preds
from sklift.models import SoloModel
from catboost import CatBoostClassifier

cat_features = ['history']
sm = SoloModel(CatBoostClassifier(iterations=20, thread_count=1, random_state=42, silent=True))
sm = sm.fit(X_train, y_train, estimator_fit_params={'cat_features': cat_features})


sm_score = uplift_at_k(y_true=y_train, uplift=uplift_sm, strategy='by_group', k=0.3)
print(f'uplift@30%: {sm_score:.4f}')

models_results['approach'].append('SoloModel')
models_results['uplift@30%'].append(sm_score)

# Получим условные вероятности выполнения целевого действия при взаимодействии для каждого объекта
sm_trmnt_preds = sm.trmnt_preds_
# И условные вероятности выполнения целевого действия без взаимодействия для каждого объекта
sm_ctrl_preds = sm.ctrl_preds_

# Отрисуем распределения вероятностей и их разность (uplift)
plot_uplift_preds(trmnt_preds=sm_trmnt_preds, ctrl_preds=sm_ctrl_preds);

TypeError: fit() missing 1 required positional argument: 'treatment'